In [123]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

<center><h2><b>Leer DB</b></h2></center>

In [199]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

/tmp/ipykernel_4991/2990043772.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [200]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

<center><h2><b>admissionDx</b></h2></center>

In [214]:
dfs['admissionDx']

,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,2900423,162,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
1,2900423,162,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No,No,No
2,2900423,162,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Sepsis, pulmonary","Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,944,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,944,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
...,...,...,...,...,...
7573,2898513,50,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Gastrointestinal|Encephalopathy, hepatic","Encephalopathy, hepatic","Encephalopathy, hepatic"
7574,2899929,11,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes,Yes,Yes
7575,2899929,11,admission diagnosis|Elective|Yes,Yes,Yes
7576,2899929,11,"admission diagnosis|All Diagnosis|Operative|Diagnosis|Genitourinary|TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy"


In [215]:
# Drop columns
dfs['admissionDx'] = dfs['admissionDx'].drop(columns=[
    'admitdxenteredoffset', # Preguntar
    'admitdxtext', # En la gran mayoría de los casos es igual a admitdxname
], axis=1)

In [264]:
#
# Nota: Es una mierda porque tiene mucho más sentido con onehot, pero hay un huevo de diagnósticos y como 10~20 notas adicionales
# TODO: diag1 = 4 ??
#

# admitdxpath - admitdxname: Cada patientunistayid tiene n filas relacionadas con todos las diagnosis encontradas.
def parse_admitdx(group):
    pusID, group_df = group
    diagnosis = []
    additionalinfo = []
    
    for i, row in group_df.iterrows():
        if row['admitdxname'] in ('No', 'Yes'):
            path_parts = row['admitdxpath'].split('|')
            mode = path_parts[-2]
            
            additionalinfo.append(mode + ' ' + row['admitdxname'])
        else:
            diagnosis.append(row['admitdxname'])
            
    # Rellenar 3 huecos en diagnosis y 2 en additionalinfo
    diagnosis = (sorted(diagnosis) + ['No' for _ in range(3)])[:3]
    additionalinfo = (sorted(additionalinfo) + ['No' for _ in range(3)])[:3]
    
    return [pusID] + diagnosis + additionalinfo

new_df = pd.DataFrame(columns=['patientunitstayid', 'diag1', 'diag2', 'diag3', 'additional1', 'additional2', 'additional3'])
for i,  group in enumerate(dfs['admissionDx'].groupby('patientunitstayid')):
    row = parse_admitdx(group)
    new_df.loc[i] = row

new_df

,patientunitstayid,diag1,diag2,diag3,additional1,additional2,additional3
0,141765,Cardiovascular,"Rhythm disturbance (atrial, supraventricular)",No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No
1,143870,Cardiovascular,"Endarterectomy, carotid",No,Elective No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
2,144815,Neurologic,"Overdose, other toxin, poison or drug",No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No
3,145427,"GI perforation/rupture, surgery for",Gastrointestinal,No,Elective No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
4,147307,Cardiovascular,"Endarterectomy, carotid",No,Elective Yes,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
...,...,...,...,...,...,...,...
2216,3351763,"Fistula/abscess, surgery for (not inflammatory bowel disease)",Gastrointestinal,No,Elective Yes,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? Yes,No
2217,3352230,4,"CABG alone, coronary artery bypass grafting",Cardiovascular,Elective Yes,Internal mammary artery graft? Yes,Pre-op MI during current hospitalization Yes
2218,3352231,Cardiovascular,"Infarction, acute myocardial (MI)",inferior,Thrombolytic Therapy received within 24 hours No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No
2219,3352333,GI obstruction,Gastrointestinal,No,Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission? No,No,No


In [232]:
dfs['admissionDx']

,patientunitstayid,admitdxpath,admitdxname
0,2900423,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular
1,2900423,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No,No
2,2900423,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular
...,...,...,...
7573,2898513,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Gastrointestinal|Encephalopathy, hepatic","Encephalopathy, hepatic"
7574,2899929,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes,Yes
7575,2899929,admission diagnosis|Elective|Yes,Yes
7576,2899929,"admission diagnosis|All Diagnosis|Operative|Diagnosis|Genitourinary|TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy"


<center><h2><b>allergy</b></h2></center>

In [201]:
dfs['allergy']

,patientunitstayid,allergyoffset,allergyenteredoffset,allergynotetype,specialtytype,usertype,rxincluded,writtenineicu,drugname,allergytype,allergyname,drughiclseqno
0,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
1,243097,1288,1294,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
2,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
3,243097,21,28,Admission,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
4,243097,3988,3989,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2470,3351763,14,51,Admission,eCM Primary,THC Nurse,False,True,NaN,Non Drug,latex,NaN
2471,3351763,14,51,Admission,eCM Primary,THC Nurse,False,True,LEVAQUIN,Drug,LEVAQUIN,12383.0
2472,3353113,77,86,Admission,eCM Primary,THC Nurse,False,True,NaN,Non Drug,Contrast Dye,NaN
2473,3353113,77,86,Admission,eCM Primary,THC Nurse,False,True,PENICILLIN G BENZATHINE,Drug,PENICILLIN G BENZATHINE,3941.0


In [203]:
dfs['allergy'] = dfs['allergy'].drop(columns=[
    'allergyoffset', # Preguntar
    'allergyenteredoffset', # Preguntar
    'specialtytype', # Poca varianza 2453 – 22
    'drughiclseqno'
], axis=1)

In [209]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_colwidth', None)
#sql_query('SELECT * FROM allergy GROUP BY drugname, allergytype').head(400)

dfs['allergy']['drugname'].isna().value_counts()

False    1480
True      995
Name: drugname, dtype: int64